# Import libraries

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from libraries.dataManipulation import *
from libraries.gridManipulation import *
from data_preparator import *
import numpy as np
from collections import Counter

# Create and plot grid

In [ ]:
#GRID CONFIGURATION IS DECLARED
o_x = 5000
o_y = -10000
o = (o_x,o_y)

#TOTAL GRID SHOULD BE SIZE 67000x13000
patch_width = 1000
patch_height = 1000
patch_size = (patch_width, patch_height)

total_patches = (67,14)
abs_total_patches = total_patches[0]*total_patches[1]

grid_angle = 0

In [ ]:
#GRID IS CREATED USING CONFIG
grid = create_grid(o, patch_size, total_patches, grid_angle)
#GRID IS PLOTTED ON STATION
plot_grid(grid, 'ehv')

# Get data and separate by direction and train presence

In [ ]:
%%time
%%capture
data = getDay(20220314,'ehv')
data = data.drop(data[data.x_pos < o_x].index)
data = data.drop(data[data.y_pos < o_y].index)
data = data.drop(data[data.x_pos > (total_patches[0]-1)*patch_width].index)
data = data.drop(data[data.y_pos > (total_patches[1]-1)*patch_height].index)

In [ ]:
data

In [ ]:
%%time
%%capture
(data_on_array ,data_off) = data_division(data)
#data_on_array = [data_on_no_train, data_on_train_top, data_on_train_bottom, data_on_train_both]

In [ ]:
data_on_no_train = data_on_array[0]
data_on_train_top = data_on_array[1]
data_on_train_bottom = data_on_array[2]
data_on_train_both = data_on_array[3]

In [ ]:
data_off

# Frequency calculation

Now that the data has been prepared, and divided into 5 different dataframes, the data will be processed, and one matrix will be created out of each dataframe.
For all of the observed day data so far, there are no moments where 2 trains are in the platform at the same time, so a matrix for this situation cannot be created. For that reason only the following matrices will be computed:
- Transition matrix for offboarding
- Transition matrix for onboarding without a train in the platform
- Transition matrix for onboarding with a train at the top of the platform
- Transition matrix for onboarding with a train at the bottom of the platform

The algorythm for the computation of the matrix should be the same for all 4 dataframe, so I will be using the offboarding data for development.

In [ ]:
def zone(x_pos, y_pos):
    origin = o
    size = patch_size
    
    i = np.floor((x_pos-origin[0])/size[0])
    j = np.floor((y_pos-origin[1])/size[1])
    
    total_x_patches = total_patches[0]
    total_y_patches = total_patches[1]
    
    zone = (i*total_y_patches)+j
    return int(zone)

In [ ]:
%%time
print('Processing data_off...')
data_off['zone'] = np.vectorize(zone)(data_off['x_pos'], data_off['y_pos'])
print('...data_off processed.')

print('Processing data_on_no_train...')
data_on_no_train['zone'] = np.vectorize(zone)(data_on_no_train['x_pos'], data_on_no_train['y_pos'])
print('...data_on_no_train processed.')

print('Processing data_on_train_top...')
data_on_train_top['zone'] = np.vectorize(zone)(data_on_train_top['x_pos'], data_on_train_top['y_pos'])
print('...data_on_train_top processed.')

print('Processing data_on_train_bottom...')
data_on_train_bottom['zone'] = np.vectorize(zone)(data_on_train_bottom['x_pos'], data_on_train_bottom['y_pos'])
print('...data_on_train_bottom processed.')

In [ ]:
%%time
people_list = data_off.tracked_object.unique()

#Compute zone count
zone_count = data_off['zone'].value_counts().to_dict()

def compute_transition_counts(person, df):
    # Create a copy of the dataframe before modifying it
    person_moves = df[df['tracked_object']==person].copy()
    # Use .loc to select the 'zone' column and assign the shifted values
    person_moves.loc[:, 'next_zone'] = person_moves['zone'].shift(periods=-1)
    person_moves = person_moves.fillna(-1)
    person_moves['next_zone'] = person_moves['next_zone'].astype(int)
    person_moves['transition'] = list(zip(person_moves['zone'], person_moves['next_zone']))
    transition_count_series = person_moves.groupby('transition').size()
    transition_counts_dict = transition_counts.to_dict()
    return transition_counts_dict

transition_counts_total = {}
people_done = 0
for person in people_list:
    print(people_done*100/len(people_list))
    transition_counts_total = Counter(transition_counts_total) + Counter(compute_transition_counts(person, data_off))
    people_done+=1

0.0
0.004916420845624385
0.00983284169124877
0.014749262536873156
0.01966568338249754
0.024582104228121928
0.029498525073746312
0.0344149459193707
0.03933136676499508
0.04424778761061947
0.049164208456243856
0.05408062930186824
0.058997050147492625
0.063913470993117
0.0688298918387414
0.07374631268436578
0.07866273352999016
0.08357915437561456
0.08849557522123894
0.09341199606686332
0.09832841691248771
0.10324483775811209
0.10816125860373647
0.11307767944936087
0.11799410029498525
0.12291052114060963
0.127826941986234
0.13274336283185842
0.1376597836774828
0.14257620452310718
0.14749262536873156
0.15240904621435594
0.15732546705998032
0.16224188790560473
0.1671583087512291
0.1720747295968535
0.17699115044247787
0.18190757128810225
0.18682399213372664
0.19174041297935104
0.19665683382497542
0.2015732546705998
0.20648967551622419
0.21140609636184857
0.21632251720747295
0.22123893805309736
0.22615535889872174
0.23107177974434612
0.2359882005899705
0.24090462143559488
0.24582104228121926
0

2.1337266470009832
2.1386430678466075
2.1435594886922322
2.1484759095378565
2.1533923303834808
2.158308751229105
2.1632251720747298
2.168141592920354
2.1730580137659783
2.1779744346116026
2.1828908554572273
2.1878072763028515
2.192723697148476
2.1976401179941
2.202556538839725
2.207472959685349
2.2123893805309733
2.217305801376598
2.2222222222222223
2.2271386430678466
2.232055063913471
2.2369714847590956
2.24188790560472
2.246804326450344
2.2517207472959684
2.256637168141593
2.2615535889872174
2.2664700098328416
2.271386430678466
2.2763028515240906
2.281219272369715
2.286135693215339
2.2910521140609634
2.295968534906588
2.3008849557522124
2.3058013765978367
2.310717797443461
2.3156342182890857
2.32055063913471
2.325467059980334
2.330383480825959
2.335299901671583
2.3402163225172075
2.3451327433628317
2.3500491642084564
2.3549655850540807
2.359882005899705
2.3647984267453293
2.369714847590954
2.3746312684365782
2.3795476892822025
2.3844641101278268
2.3893805309734515
2.3942969518190758


4.321533923303835
4.3264503441494595
4.331366764995083
4.336283185840708
4.341199606686333
4.346116027531957
4.351032448377581
4.355948869223205
4.36086529006883
4.3657817109144545
4.370698131760078
4.375614552605703
4.380530973451328
4.385447394296952
4.390363815142576
4.3952802359882
4.400196656833825
4.40511307767945
4.410029498525073
4.414945919370698
4.419862340216323
4.424778761061947
4.429695181907571
4.434611602753196
4.43952802359882
4.444444444444445
4.4493608652900685
4.454277286135693
4.459193706981318
4.464110127826942
4.469026548672566
4.473942969518191
4.478859390363815
4.48377581120944
4.4886922320550635
4.493608652900688
4.498525073746313
4.503441494591937
4.5083579154375615
4.513274336283186
4.51819075712881
4.523107177974435
4.528023598820059
4.532940019665683
4.537856440511308
4.542772861356932
4.5476892822025565
4.552605703048181
4.557522123893805
4.56243854473943
4.5673549655850545
4.572271386430678
4.577187807276303
4.582104228121927
4.587020648967552
4.591937069

6.568338249754179
6.573254670599804
6.578171091445427
6.583087512291052
6.588003933136677
6.592920353982301
6.597836774827925
6.602753195673549
6.607669616519174
6.612586037364799
6.6175024582104225
6.622418879056047
6.627335299901672
6.632251720747296
6.6371681415929205
6.642084562438545
6.647000983284169
6.651917404129794
6.6568338249754175
6.661750245821042
6.666666666666667
6.671583087512291
6.6764995083579155
6.68141592920354
6.686332350049164
6.691248770894789
6.696165191740413
6.701081612586037
6.705998033431662
6.710914454277286
6.7158308751229105
6.720747295968535
6.725663716814159
6.730580137659784
6.735496558505408
6.740412979351032
6.745329400196657
6.750245821042281
6.755162241887906
6.76007866273353
6.764995083579154
6.769911504424779
6.774827925270404
6.779744346116027
6.784660766961652
6.789577187807276
6.794493608652901
6.799410029498525
6.804326450344149
6.809242871189774
6.814159292035399
6.819075712881022
6.823992133726647
6.828908554572271
6.833824975417896
6.83874

8.8102261553589
8.815142576204524
8.820058997050147
8.824975417895772
8.829891838741396
8.834808259587021
8.839724680432646
8.844641101278269
8.849557522123893
8.854473942969518
8.859390363815143
8.864306784660767
8.869223205506392
8.874139626352015
8.87905604719764
8.883972468043265
8.88888888888889
8.893805309734514
8.898721730580137
8.903638151425762
8.908554572271386
8.913470993117011
8.918387413962636
8.923303834808259
8.928220255653883
8.933136676499508
8.938053097345133
8.942969518190758
8.947885939036382
8.952802359882005
8.95771878072763
8.962635201573255
8.96755162241888
8.972468043264504
8.977384464110127
8.982300884955752
8.987217305801376
8.992133726647001
8.997050147492626
9.00196656833825
9.006882989183874
9.011799410029498
9.016715830875123
9.021632251720748
9.026548672566372
9.031465093411995
9.03638151425762
9.041297935103245
9.04621435594887
9.051130776794494
9.056047197640117
9.060963618485742
9.065880039331367
9.070796460176991
9.075712881022616
9.08062930186824
9.